In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM,TrainingArguments,LlamaTokenizer
import torch
output_dir="/home/flowers/work/hf/deepseek-coder-1.3b-instruct"
type_use=torch.bfloat16
tokenizer = AutoTokenizer.from_pretrained(output_dir,local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(
    output_dir,
    torch_dtype=type_use,

    # quantization_config=quantization_config,
    device_map="auto",
    local_files_only=True
)
tokenizer.padding_side='left'
tokenizer.pad_token = tokenizer.eos_token
model.eval()
model.config.use_cache = True


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
prompt="""You are an AI programming assistant, utilizing the DeepSeek Coder model, developed by DeepSeek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer.
### Instruction:
{instruction}
### Response:
"""
prompt_eval="You need to write the entire code for the following problem in a code blocks (i.e.,within “```”):\n"

In [16]:
# from evalplus.data import get_human_eval_plus, write_jsonl

# # len(get_human_eval_plus())
# dic_puzzles = get_human_eval_plus()
from evalplus.data import get_mbpp_plus, write_jsonl
dic_puzzles=get_mbpp_plus()

list_keys= list(dic_puzzles.keys())
list_task_id = [dic_puzzles[key]["task_id"] for key in list(dic_puzzles.keys())]
list_testset = [dic_puzzles[key]["prompt"] for key in list(dic_puzzles.keys())]


In [17]:
list_prompt=[prompt.format(instruction=prompt_eval+i) for i in list_testset[:2]]
with torch.inference_mode():

    inputs = tokenizer(list_prompt, return_tensors="pt",padding=True).to("cuda")

    outputs = model.generate(**inputs,max_new_tokens=512,do_sample=True, temperature=0.7)
    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)


Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


In [18]:
for i in generated_texts:
    print(i)
    print("===\n\n")    

You are an AI programming assistant, utilizing the DeepSeek Coder model, developed by DeepSeek Company, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer.
### Instruction:
You need to write the entire code for the following problem in a code blocks (i.e.,within “```”):
"""
Write a function to find the shared elements from the given two lists.
assert set(similar_elements((3, 4, 5, 6),(5, 7, 4, 10))) == set((4, 5))
"""

### Response:
Here is the Python code that solves the problem:

```python
def similar_elements(list1, list2):
    # Convert lists to sets
    set1 = set(list1)
    set2 = set(list2)
    
    # Find the intersection of sets
    common = set1 & set2
    
    # Convert the resulting set back to a list
    common_list = list(common)
    
    return common_list

# Test the function
print(similar_elements((3, 4, 5, 6),(5, 7, 4, 10)))
```